# Emotion Detection with MediaPipe

**First execute `pip install requirements.txt` in command line in order to setup the environment.
Then, import the libraries.**

In [1]:
import mediapipe as mp
import cv2 
import csv
import os
import numpy as np
import pandas as pd
import pickle
import time
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from PIL import ImageFont, ImageDraw, Image

In [2]:
mpDraw = mp.solutions.drawing_utils
mediapipeHolistic = mp.solutions.holistic

In [3]:
# mediapipe testing
# press q to quit
cap = cv2.VideoCapture(0)

with mediapipeHolistic.Holistic(min_detection_confidence=0.01, min_tracking_confidence=0.01) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        results = holistic.process(image)
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
       
        # Face
        mpDraw.draw_landmarks(image, results.face_landmarks, mediapipeHolistic.FACE_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(255, 160, 122), thickness=1, circle_radius=1),
                                 mpDraw.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                                 )

        # Pose
        mpDraw.draw_landmarks(image, results.pose_landmarks, mediapipeHolistic.POSE_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(255, 160, 122),  circle_radius=4),
                                 mpDraw.DrawingSpec(color=(121,44,250),  circle_radius=2)
                                   )

        cv2.imshow('Test', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

**We create our dataset of videos with following codes.**

In [4]:
coordinates = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
coordinates

501

In [5]:
landmarks = ['class']
for coordinates_number in range(1, coordinates+1):
    landmarks += ['x{}'.format(coordinates_number), 'y{}'.format(coordinates_number), 'z{}'.format(coordinates_number), 'v{}'.format(coordinates_number)]

In [ ]:
with open('detection.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [6]:
# class you want to train for 
# "Neutral" is for example
trainName = "Neutral"
   
cap = cv2.VideoCapture(0)

with mediapipeHolistic.Holistic(min_detection_confidence=0.01, min_tracking_confidence=0.01) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret==False:
             break
       
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)        
     
        results = holistic.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Face
        mpDraw.draw_landmarks(image, results.face_landmarks, mediapipeHolistic.FACE_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(255, 160, 122), thickness=1, circle_radius=1),
                                 mpDraw.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                                 )

        # Pose
        mpDraw.draw_landmarks(image, results.pose_landmarks, mediapipeHolistic.POSE_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(255, 160, 122),  circle_radius=4),
                                 mpDraw.DrawingSpec(color=(121,44,250),  circle_radius=2)
                                   )

        try:
            coordinate_pose = results.pose_landmarks.landmark
            seqPose = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in coordinate_pose]).flatten())
            
            coordinate_face = results.face_landmarks.landmark
            seqFace = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in coordinate_face]).flatten())

            line = seqPose+seqFace

            line.insert(0, trainName)

            with open('detection.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(line) 
            
        except:
            pass
                        
        cv2.imshow('Data Generation', image)

        if cv2.waitKey(10) & 0xFF == ord('c'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
csv = pd.read_csv('detection.csv', encoding='iso-8859-9')

In [8]:
X = csv.drop('class', axis=1) 
y = csv['class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [9]:
sklearnP = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=10000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
models = {}
for algo, pipeline in sklearnP.items():
    model = pipeline.fit(X_train, y_train)
    models[algo] = model
    
models['rc'].predict(X_test)

In [ ]:
for algo, model in models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
models['rf'].predict(X_test)

In [ ]:
# SAVE THE MODEL
with open('duygutespiti.pkl', 'wb') as f:
    pickle.dump(models['rf'], f)

**Following part is for running the emotion detection program using our trained model.**

In [10]:
# LOAD THE MODEL
# choose one of the following for your desired language output
modelfile = 'emotion-detection-eng.pkl' # for english
#modelfile = 'emotion-detection-tr.pkl' # for turkish
with open(modelfile, 'rb') as f:
    model = pickle.load(f)

In [11]:
# press q to quit
cap = cv2.VideoCapture(0)

with mediapipeHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        b,g,r,a = 0,0,0,0

        image = np.zeros((640,480),np.uint8)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        results = holistic.process(image)
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Extract Pose landmarks
            coordinate_pose = results.pose_landmarks.landmark
            seqPose = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in coordinate_pose]).flatten())
            
            # Extract Face landmarks
            coordinate_face = results.face_landmarks.landmark
            seqFace = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in coordinate_face]).flatten())
            
            line = seqPose+seqFace
            X = pd.DataFrame([line])
            face_pose_class = model.predict(X)[0]
            
            font = ImageFont.truetype("arial.ttf", 50)
            PILImage = Image.fromarray(image)
            draw = ImageDraw.Draw(PILImage)
            draw.text((15, 10), face_pose_class, font = font, fill = (b, g, r, a))
            image = np.array(PILImage)

        except:
            pass
                        
        cv2.imshow('Emotion Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()